In [4]:
import flwr as fl
import tensorflow as tf

In [15]:
!pip install torch torchvision

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [16]:
import torch

In [7]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 8s 0us/step


In [8]:
model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

In [13]:
class CifarClient(fl.client.NumPyClient):
    def get_parameters(self):
        return model.get_weights()

    def fit(self, parameters, config):
        model.set_weights(parameters)
        model.fit(x_train, y_train, epochs=1, batch_size=32, steps_per_epoch=3)
        return model.get_weights(), len(x_train), {}

    def evaluate(self, parameters, config):
        model.set_weights(parameters)
        loss, accuracy = model.evaluate(x_test, y_test)
        return loss, len(x_test), {"accuracy": accuracy}

In [14]:
fl.client.start_numpy_client("[::]:8080", client=CifarClient())

DEBUG flower 2021-12-29 16:47:50,248 | connection.py:36 | ChannelConnectivity.IDLE
INFO flower 2021-12-29 16:47:50,251 | app.py:61 | Opened (insecure) gRPC connection
DEBUG flower 2021-12-29 16:47:50,252 | connection.py:36 | ChannelConnectivity.TRANSIENT_FAILURE
DEBUG flower 2021-12-29 16:47:50,462 | connection.py:68 | Insecure gRPC channel closed


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses"
	debug_error_string = "{"created":"@1640792870.250000000","description":"Failed to pick subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":4143,"referenced_errors":[{"created":"@1640792870.250000000","description":"failed to connect to all addresses","file":"src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc","file_line":398,"grpc_status":14}]}"
>